## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
city_data_to_load = os.path.join("../Weather_Database", "WeatherPy_Database.csv")

#Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,54.90,58,4,8.21,clear sky
1,1,Buraydah,SA,26.3260,43.9750,88.25,19,67,9.40,broken clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,24.64,58,20,10.36,few clouds
3,3,Coron,PH,11.9986,120.2043,88.14,60,96,3.44,overcast clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,22.55,81,100,3.53,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Buraydah,SA,26.3260,43.9750,88.25,19,67,9.40,broken clouds
3,3,Coron,PH,11.9986,120.2043,88.14,60,96,3.44,overcast clouds
16,16,Sur,OM,22.5667,59.5289,88.30,46,1,6.80,clear sky
28,28,Howard Springs,AU,-12.4970,131.0470,86.83,47,7,5.79,clear sky
29,29,Faanui,PF,-16.4833,-151.7500,81.55,69,0,11.45,clear sky
34,34,Payo,ID,-3.7508,103.6372,83.08,68,62,0.89,broken clouds
41,41,Avarua,CK,-21.2078,-159.7750,80.65,65,20,13.80,few clouds
43,43,Georgetown,MY,5.4112,100.3354,84.88,75,20,2.30,few clouds
48,48,Atuona,PF,-9.8000,-139.0333,80.02,71,18,17.34,few clouds
61,61,Kavaratti,IN,10.5669,72.6420,82.15,75,100,8.03,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna( )
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buraydah,SA,88.25,broken clouds,26.3260,43.9750,
3,Coron,PH,88.14,overcast clouds,11.9986,120.2043,
16,Sur,OM,88.30,clear sky,22.5667,59.5289,
28,Howard Springs,AU,86.83,clear sky,-12.4970,131.0470,
29,Faanui,PF,81.55,clear sky,-16.4833,-151.7500,
34,Payo,ID,83.08,broken clouds,-3.7508,103.6372,
41,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
43,Georgetown,MY,84.88,few clouds,5.4112,100.3354,
48,Atuona,PF,80.02,few clouds,-9.8000,-139.0333,
61,Kavaratti,IN,82.15,overcast clouds,10.5669,72.6420,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [34]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buraydah,SA,88.25,broken clouds,26.3260,43.9750,فندق راحة السلام
3,Coron,PH,88.14,overcast clouds,11.9986,120.2043,Coron Westown Resort
16,Sur,OM,88.30,clear sky,22.5667,59.5289,Sur Plaza Hotel
28,Howard Springs,AU,86.83,clear sky,-12.4970,131.0470,buffalo beach hostel
29,Faanui,PF,81.55,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
34,Payo,ID,83.08,broken clouds,-3.7508,103.6372,Mess keban agung
41,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
43,Georgetown,MY,84.88,few clouds,5.4112,100.3354,Cititel Penang
48,Atuona,PF,80.02,few clouds,-9.8000,-139.0333,Villa Enata
61,Kavaratti,IN,82.15,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands


In [35]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))